# First-Class Functions

Functions in Python are first-class objects.

* Created at runtime
* Assigned to a variable or element in a data structure
* Passed as an argument to a function
* Returned as the result of a function

In [28]:
import pprint

pp = pprint.PrettyPrinter()

## Treating a Function Like an Object

In [2]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)

In [3]:
print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


We can assign a function object to a variable and pass it to a function as an argument.

In [4]:
fact = factorial
print(fact)

print(fact(5))

list(map(factorial, range(11)))

<function factorial at 0x7fccdf55ebf8>
120


[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-Order Functions

A function that takes a function as argument or returns a function as the result is a _higher-order function_.

In [5]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

### Modern Replacements for map, filter, and reduce

`map` and `filter` are not important due to the introduction of list comprehensions and generator expressions.

In [6]:
print(list(map(fact, range(6))))
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))

[1, 1, 2, 6, 24, 120]
[1, 6, 120]


reduce in Python 3 is in `functools` module. Common reducing built-ins
* sum
* all
* any

In [7]:
from functools import reduce
from operator import add
reduce(add, range(100))

4950

### Anonymous Functions

`lambda` creates an anonymous function.

In [8]:
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## The Seven Flavors of Callable Objects

* User-defined functions. `def` or `lambda`
* Built-in functions
* Built-in methods
* Methods
* Classes: When invoked, a class runs its `__new__` method to create an instance, then `__init__` to initialize it and finally the instance is returned to the caller.
* Class instances: If a class defines a `__call__` method, then its instances may be invoked as functions.
* Generator functions. Functions or methods that use the `yield` keyword.

In [9]:
# use the callable to determine whether an object is callable
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## User-Defined Callable Types

In [10]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        return self.pick()

In [11]:
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
print(callable(bingo))

2
0
True


## Function Introspection

Use `dir` to view the attributes of function object

In [12]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

The `__dict__` attribute stores user attributes assigned to the function. It can be used for annotation.

In [14]:
def upper_case_name(obj):
    return "{} {}".format(obj.first_name, obj.last_name).upper()

upper_case_name.short_description = "Customer name"
print(upper_case_name.__dict__["short_description"])

Customer name


What are the attributes that are specific to functions?


In [22]:
class C: pass
obj = C()
def func(): pass
attris = sorted(set(dir(func)) - set(dir(obj)))
print('attributes in function objects but not in class objects', attris)

attributes in function objects but not in class objects ['__annotations__', '__call__', '__closure__', '__code__', '__defaults__', '__get__', '__globals__', '__kwdefaults__', '__name__', '__qualname__']


In [34]:
for attri in attris:
    if attri == '__globals__':
        continue
    pp.pprint(f'attri:{attri}, val: {getattr(func, attri)}')

'attri:__annotations__, val: {}'
("attri:__call__, val: <method-wrapper '__call__' of function object at "
 '0x7fccdf500400>')
'attri:__closure__, val: None'
('attri:__code__, val: <code object func at 0x7fccdf55ded0, file '
 '"<ipython-input-22-49668ee95474>", line 3>')
'attri:__defaults__, val: None'
("attri:__get__, val: <method-wrapper '__get__' of function object at "
 '0x7fccdf500400>')
'attri:__kwdefaults__, val: None'
'attri:__name__, val: func'
'attri:__qualname__, val: func'


## From Positional to Keyword-Only Parameters

Python function provides the extremely flexible parameter handling mechanism, enhanced with keyword-only arguments.

In [47]:
# tag generates HTML
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join(f'<{name}{attr_str}>{c}</{name}>' for c in content)
    else:
        return f'<{name}{attr_str} />'

In [48]:
print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name='img'))
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />


'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

**New feature Py3** Keyword-only arguments. `cls` can only be given as a keyword argument. To specify keyword-only arguments, name them after the argument prefixed with `*`. If you don't want to support variable psoitional arguments, put a `*` by itself (anonymous).

In [49]:
def f(a, * ,b):
    return a, b

f(1, b=2)

(1, 2)

## Retrieving Information About Parameters

In [54]:
def clip(text, max_len=80):
    """
    Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [56]:
print('clip default:', clip.__defaults__)
print('clip code:', clip.__code__)
print('clip variables:', clip.__code__.co_varnames)
print('clip arg count:', clip.__code__.co_argcount)
print('clip args:', clip.__code__.co_varnames[:clip.__code__.co_argcount])

clip default: (80,)
clip code: <code object clip at 0x7fccdf5028a0, file "<ipython-input-54-269effde0f09>", line 1>
clip variables: ('text', 'max_len', 'end', 'space_before', 'space_after')
clip arg count: 2
clip args: ('text', 'max_len')


`inspect` module provide a better way to extract the information of functions

In [59]:
from inspect import signature
sig = signature(clip)
print('sig:', sig)

for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

sig: (text, max_len=80)
POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


An `inspect.Signature` can bind arguments to formal parameters via the `bind` method.

In [61]:
sig = signature(tag)
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
for name, value in bound_args.arguments.items():
    print(name, '=', value)

del my_tag['name']
bound_args = sig.bind(**my_tag)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


TypeError: missing a required argument: 'name'

## Function Annotations

Py3 provides syntax to annotate types on parameters and return values.

In [63]:
def clip_annot(text: str, max_len: int=80) -> str:
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

print(clip_annot.__annotations__)

{'text': <class 'str'>, 'max_len': <class 'int'>, 'return': <class 'str'>}


In [64]:
sig = signature(clip_annot)
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
<class 'int'> : max_len = 80


## Packages for Functional Programming

Two useful packages

### The operator Module

* operators like `mul`
* pick items: `itemgetter`
* read attributes: `attrgetter`

In [68]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1), 1)

for n in range(10):
    print(f'{n}!={fact(n)}')

0!=1
1!=1
2!=2
3!=6
4!=24
5!=120
6!=720
7!=5040
8!=40320
9!=362880


In [69]:
metro_data = [
('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


`itemgetter(1)` creates a function pick the second element of any objects that implement `__getitem__`.
Pass multiple index arguments to `itemgetter` creates a function returning tuples.

In [71]:
for city in metro_data:
    print(itemgetter(1, 0)(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


`attrgetter` creates functions that read object attributes by name.

In [73]:
from collections import namedtuple
from operator import attrgetter
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) for name, cc, pop, (lat, long) in metro_data]

name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


`methodcaller` creates a function by binding extra arguments

In [76]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
print('upcase(s)=', upcase(s))

hiphenate = methodcaller('replace', ' ', '-')
print('hiphenate(s)=', hiphenate(s))

upcase(s)= THE TIME HAS COME
hiphenate(s)= The-time-has-come


### Freezing Arguments with functools.partial
Similar to `methodcaller`, `functools.partial` is a high-order function that allows paritial application of a function. It takes a callable as first argument, followed by an arbitrary number of positional and keyword arguments to bind.

In [77]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [82]:
picture = partial(tag, 'img', cls='pic-frame')
print(picture(src='wumpus.jpeg'))
print('picture def:', picture)
print('picture bound func:', picture.func)
print('picture args:', picture.args)
print('picture keywords:', picture.keywords)

<img class="pic-frame" src="wumpus.jpeg" />
picture def: functools.partial(<function tag at 0x7fccdf500c80>, 'img', cls='pic-frame')
picture bound func: <function tag at 0x7fccdf500c80>
picture args: ('img',)
picture keywords: {'cls': 'pic-frame'}
